In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium

In [2]:
%matplotlib inline

plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (16,8)
plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = (16,8)

In [3]:
## Calculating distance (km) between two cordinates
def distance(lat1, long1, lat2, long2):
    #Radius in km
    R = 6371

    lat1=np.deg2rad(lat1)
    lat2=np.deg2rad(lat2)
    long1=np.deg2rad(long1)
    long2=np.deg2rad(long2)
    dlong = long2 - long1
    dlat = lat2 - lat1

    a = np.power((np.sin(dlat/2)), 2) + np.cos(lat1) * np.cos(lat2) * np.power(np.sin(dlong/2), 2)
    c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a) )
    d = R * c #(where R is the radius of the Earth)
    return d


-------------------------------------------------------

# Data preparation and description

Let's start by simply importing the datasets and doing some initial restructuring

In [4]:
# importing data
df_rentals = pd.read_csv("Data/Rentals_2019-4-2_1456.csv", parse_dates=[0,1])
df_hubav = pd.read_csv("Data/Hubavailabilityaudits_2019-4-2_1535.csv", parse_dates=[3])
df_hubs = pd.read_csv("Data/Hubs_2019-4-2_1201.csv", parse_dates=[0,-1])
df_searchlog1 = pd.read_csv("Data/Searchlogs_2019-4-2_1234.csv", parse_dates=[-1])
df_searchlog2 = pd.read_csv("Data/Searchlogs_2019-4-2_1504.csv", parse_dates=[-1])

FileNotFoundError: [Errno 2] File b'Data/Rentals_2019-4-2_1456.csv' does not exist: b'Data/Rentals_2019-4-2_1456.csv'

In [ ]:
#concatenating searchlogs dataframes (the second file is a continuation of the first)
df_searchlog = pd.concat([df_searchlog1, df_searchlog2])

-----------------
## Overview

The data we will be dealing with belongs to Donkey Republic, a bike-sharing company operating in Copenhagen. The service offered by Donkey Republic falls under the family of *hub-based* bike-sharing services. This means that bikes can only be picked-up and dropped-off in specific areas - i.e. "virtual" areas (meaning that no physical platforms are installed) called hubs. Our goal will be to define a model for the prediction of bike usage demand in the various hubs. Having accurate predictions of demand will be the key element in order to take coherent decisions on the supply side (supply dimensioning, rebalancing, hub positioning, etc.).

Our dataset will be constituted by various measurements of the bike usage and of the service provider. The information available to us is organized in the following way:

1) *df_rentals*: represents the actual bike rentals indicating time and hub for both pick-up and drop-off, together with the user id associated with that rental

In [ ]:
df_rentals.head(10)

2) *df_hubs*: describes various characteristics for each hub such as time of creation, geo-spatial location, id, name and (eventual) time of deletion

In [ ]:
df_hubs.head(10)

3) *df_searchlog*: represents the geo-localization of users in the moment they access the app. For every search it describes location, time and user_id 

In [ ]:
df_searchlog.head(10)

4) *df_hubav*: represents the availability of bikes for every hub and how it evolved with time. In particular, at a certain timestamp "created_at", the bike count in a specific hub goes from the value in "bike_count_from" to the one in "bike_count_to". To give an example, if at a certain timestamp there has been a drop-off, and the previous availability of bikes was 8 (bike_count_from), then the new availability would be 9 (bike_count_to). 

This table also shows values for optimal and maximum capacity of bikes for every hub. It is already possible to notice some anamalous values in the data (i.e. 0 maximum capacity) which definitely need some further inspection...

In [ ]:
df_hubav.head(10)

Before going in more datail, here are some first general descriptions for the our four tables:

In [ ]:
df_rentals.describe()

In [ ]:
df_hubs.describe()

In [ ]:
df_searchlog.describe()

In [ ]:
df_hubav.describe()

----------------------------------

## Hubs

In what follows we aim at visualizing on the map of Copenhagen all the existing (and deleted) hubs in our dataset. In order to do this we first need to differentiate between the still existing-hubs and the deleted ones:

In [ ]:
# Centered location (latitude, longitude) which we will use in all our following plots
initial_location = [55.6775757, 12.579571639999999]

In [ ]:
df_deleted_hubs = df_hubs.where(pd.notna(df_hubs.deleted_at) == True).dropna()

In [ ]:
df_deleted_hubs.head(10)

In [ ]:
df_existing_hubs = df_hubs.where(pd.notna(df_hubs.deleted_at) == False).dropna(how='all')

In [ ]:
df_existing_hubs.head(10)

Ok, so now we are ready to plot the hubs on the map differentiating the existing (in blue) from the deleted (in red) 

In [ ]:
# Getting latitude-longitude locations for all the hubs
existing_locations = df_existing_hubs.iloc[:, 1:3].values.tolist()
del_hub_locations = df_deleted_hubs.iloc[:, 1:3].values.tolist()

np.array(existing_locations).shape

In [ ]:
from sklearn.cluster import KMeans

n_superhubs = 100
margin = 0.01

# Preprocessing (flip and rotate 90deg clockwise (mirror in y=x, change x by y))
existing_locations_arr = np.array(existing_locations)
existing_locations_arr[:,[0,1]] = existing_locations_arr[:,[1,0]]


kmeans = KMeans(init='k-means++', n_clusters=n_superhubs, n_init=10)
kmeans.fit(existing_locations_arr)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .0005     # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = existing_locations_arr[:, 0].min() - margin, existing_locations_arr[:, 0].max() + margin
y_min, y_max = existing_locations_arr[:, 1].min() - margin, existing_locations_arr[:, 1].max() + margin
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
print(Z.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.terrain,
           aspect='equal', origin='lower')

plt.plot(existing_locations_arr[:, 0], existing_locations_arr[:, 1], 'k.', markersize=2)

In [ ]:
hub_map = folium.Map(location=initial_location, zoom_start=13)

for point in range(0, len(existing_locations)):
    folium.CircleMarker(
    location=existing_locations[point],
    radius=3,
    popup='Hub_{}'.format(df_existing_hubs.iloc[point, 3]),
    color='#363636',
    fill=True,
    fill_color='#363636').add_to(hub_map)
    
'''for point in range(0, len(del_hub_locations)):
    folium.CircleMarker(
    location=del_hub_locations[point],
    radius=3,
    popup='Hub_{}'.format(df_deleted_hubs.iloc[point, 3]),
    color='crimson',
    fill=True,
    fill_color='crimson').add_to(hub_map)'''
    
folium.raster_layers.ImageOverlay(Z,
                                  bounds=[[yy.min(),xx.min()],[yy.max(),xx.max()]],
                                  opacity=0.5,
                                  colormap=plt.cm.prism,
                                  origin='lower').add_to(hub_map)

hub_map

<br>

-----------------

## Rentals

Being our goal the prediction of bike demand in the various hubs, the "Rentals" DataFrame definitely plays a central role. This will be the source of our observed demand in each hub. As we have seen above, this dataset is for now missing any kind of aggregation, meaning that our bike rentals are only recorded as individual instances on the timeline of our data.

This definitely leaves a lot of space for modeling: we could choose to aggregate our data at different levels (e.g. we could count the rentals for every hour/day/week/month). Obviously this depends on what is the type of decisions we want to take after having extracted information from our data. In our case, the decision will mostly have (at least initially) an operational level i.e. short-medium term. Therefore we are naturally more interested in a higher granularity of our data. 

For example, the following plots regard the daily time-series of pick-ups in a specific hub (Hub #60).

(before actually being able to plot our data we must fill in a "0" value for all those days where we don't have any recorded rental, in what follows we construct a DataFrame df_hub_60 which has the complete time series of pick-ups in Hub #60)

In [ ]:
incomplete_index = pd.DatetimeIndex(pd.DatetimeIndex(df_rentals[df_rentals.pickup_hub_id == 60].created_at).date)
complete_index = pd.date_range(incomplete_index.min(),incomplete_index.max())
complete_index

The two following cells give a representation of the type of imputation we just implemented:

In [ ]:
df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d') \
.value_counts().sort_index().fillna(value=0).head(10)

In [ ]:

df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts().sort_index() \
.reindex(complete_index).fillna(value=0).head(10)

We are now ready to plot...

In [ ]:
df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts().sort_index(). \
reindex(complete_index).fillna(value=0).plot(style='-')
plt.title("Pick-up Request in Hub #60 (Norreport Area)")
plt.ylabel("Number of Pick-ups")
plt.show()

In the above plot we can observe a relevant increment in the observed demand especially during the summer (or close-to-summer) months. Other than leading us to explore some sort of seasonality this could also suggest that our data is actually generated from two distinct processes...

In the following graph we try to give a sense of this dual nature by plotting the histogram of the pickup counts (in Hub #60) by roughly trying to isolate the central part of our time-series: the "*nice weather*" data.

In [ ]:
# As a rough separation the data is devided in 3 parts (using the "official" summer dates: 21 Jun - 23 Sep)
summer_start_idx = int(np.where(df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts()
                    .sort_index().index == pd.to_datetime('2018-06-21'))[0])
summer_end_idx = int(np.where(df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts()
                       .sort_index().index == pd.to_datetime('2018-9-23'))[0])

In [ ]:
df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts().sort_index() \
.reindex(complete_index).fillna(value=0)[:summer_start_idx].hist(bins=50)
plt.title("Pre-Summer Pick-up count distribution in Hub #60 (Norreport Area)")
plt.ylabel("Frequency")
plt.xlabel("Pick-up count")
plt.xlim([0,18])
plt.show()

In [ ]:
df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts().sort_index() \
.reindex(complete_index).fillna(value=0)[summer_start_idx:summer_end_idx].hist(bins=50)
plt.title("Summer Pick-up count distribution in Hub #60 (Norreport Area)")
plt.ylabel("Frequency")
plt.xlabel("Pick-up count")
plt.xlim([0,18])
plt.show()

In [ ]:
df_rentals[df_rentals.pickup_hub_id == 60].created_at.dt.floor('d').value_counts().sort_index() \
.reindex(complete_index).fillna(value=0)[summer_end_idx:].hist(bins=50)
plt.title("Post-Summer Pick-up count distribution in Hub #60 (Norreport Area)")
plt.ylabel("Frequency")
plt.xlabel("Pick-up count")
plt.xlim([0,18])
plt.show()

It is therefore reasonable to think that our model should try to extract information from these temporal indicators in our data.

It is also important to stress the fact that these plots represent exclusively the pick-up rates for a single hub. We could have chosen to concentrate on the observation of origin-destination rentals (and so focus more on the demand of transportation from Hub A to Hub B rather than the pick-up demand in one hub).

---------------------------

## Hub availability

In the specification of our demand prediction problem it is important to notice that the realizations of demand that we observe in every hub are actually *censored*. This basically means that our variable of interest (the demand) has as upper limit the current supply of bikes. For example, suppose that we have a capacity of 10 bikes in one hub, we will never be able to observe a demand of 15, but only a maximum of 10. This behaviour will lead to an observed demand which could be an underestimate of the *real* demand (and therefore to a biased predictor)

An indicator of this censoring behaviour in our data could be observed in all those cases where the current availability of the hub reaches a value of 0. Obviously, a hub with no bikes available would not be able to satisfy any existing demand (and we would not be able therefore to observe any rentals).

The following plot represents the **minimum value** of bike availability in Hub #60 aggregated to a daily level:

(Again, the two following cells give a visual understanding of the preprocessing)

In [ ]:
# Odered representation of Hub dataset
df_hubav[df_hubav.hub_id == 60].sort_values(by='created_at').head(10)

In [ ]:
# Minimum availability for each of the sample days
df_hubav[df_hubav.hub_id == 60].groupby(pd.Grouper(key='created_at',freq='d')).min().bike_count_to.head(3)

By inspecting the data, we notice that we have missing availability records for some days. It turns out (as one would expect) that this is due to an unchanged availability in the corresponding Hub. Therefore, in this case it makes complete sense to fill in the missing days with a "*forward fill*" method:

In [ ]:
# Before fill
df_hubav[df_hubav.hub_id == 60].groupby(pd.Grouper(key='created_at',freq='d')).min().bike_count_to.head(10)

In [ ]:
# After fill
df_hubav[df_hubav.hub_id == 60].groupby(pd.Grouper(key='created_at',freq='d')).min(). \
bike_count_to.fillna(method='ffill').head(10)

We are now ready to analyse the plot:

In [ ]:
df_hubav[df_hubav.hub_id == 60].groupby(pd.Grouper(key='created_at',freq='d')).min().bike_count_to.fillna(method='ffill').plot()
plt.title("Minimum Daily Bike Availability in Hub #60 (Norreport Area)")
plt.ylabel("Minimum availability")
plt.xlabel("Day")
plt.show()

The plot clearly shows that (in this specific Hub) it is very common for the bike availability to go to 0 during the day (approximately 69% of the days)

-----------------------

## Searchlogs

We have seen how our data naturally defines a difference between *observed* and *real* demand. In this perspective, the searchlog data could represent some kind of indicator of the real demand. This data, as we described above, represents the locations of the users in the moment they access the app. It is therefore reasonable to assume that most of the access logs will represent users actually looking for an available bike nearby... 

To give a visual representation of the searchlogs the following plot represents 1000 searchlogs of 09/03/2019:

In [ ]:
df_searchlog['date'] = [d.date() for d in df_searchlog['timestamp']]
df_search_plot = df_searchlog.where(df_searchlog.date == pd.to_datetime("09/03/2019", format="%d/%m/%Y").date()).dropna()

In [ ]:
m = folium.Map(location=initial_location,
              zoom_start=13)

locations = df_search_plot.iloc[:1000, :2]
locationlist = locations.values.tolist()

for point in range(0, len(locationlist)):
    folium.CircleMarker(
    location=locationlist[point],
    radius=3,
    popup='Searchlog_{}'.format(point),
    color='#3186cc',
    fill=True,
    fill_color='#3186cc').add_to(m)
    
m

------------------

## User-specific level

Our data gives us valuable information on the rental patterns of single users. It could therefore be interesting to try and extrapolate patterns related to single individuals in our dataset. 

For example, in the following cells we first count how many rentals are recorded for every user (ordering from most active user to less active user). Then, for our plot, we select the most active user and analyse his origin-destination pattern.

In [ ]:
# Rental counts for every user
rentals_by_user = df_rentals.groupby('user_id')['user_id'].count().sort_values(ascending=False)
rentals_by_user.head(10)

In [ ]:
# Trips for user 140961 (most active in our dataset)
usr_140961 = df_rentals[df_rentals.user_id == 140961][['pickup_hub_id', 'dropoff_hub_id']].dropna()
usr_140961.head(10)

Let's also visualize the first 450 rentals on a map. In particular the blue markers represent the Pick-up locations, while the red markers represent the drop-off locations. 

Notice how: 
1. The area of Copenhagen is a pretty well defined subset of the entire city and 
2. The number of observable hubs is actually way less than 450 (this means that there is a high overlap between the plotted OD pairs - the user has several repetitive trips).

In [ ]:
user_map = folium.Map(location=initial_location,
              zoom_start=13)

locations = usr_140961.iloc[:,:]
locationlist = locations.values.tolist()

for point in range(0, len(locationlist[:450])):
    folium.CircleMarker(
    location=[df_hubs[df_hubs.id == int(locationlist[point][0])]['latitude'].values[0],df_hubs[df_hubs.id == int(locationlist[point][0])]['longitude'].values[0]],
    radius=3,
    popup='Rental_{}'.format(point),
    color='#3186cc',
    fill=True,
    fill_color='#3186cc').add_to(user_map)
    
    folium.CircleMarker(
    location=[df_hubs[df_hubs.id == int(locationlist[point][1])]['latitude'].values[0],df_hubs[df_hubs.id == int(locationlist[point][1])]['longitude'].values[0]],
    radius=3,
    popup='Rental_{}'.format(point),
    color='crimson',
    fill=True,
    fill_color='crimson').add_to(user_map)
    
user_map